In [2]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import plotly.express as px
import requests
import dash_bootstrap_components as dbc

# Fetch data from your Flask API
api_url = 'http://127.0.0.1:5000/controller_post_cleaned_reddit/api/post_cleaned_reddit_data'
response = requests.get(api_url)
data = response.json()

# Convert the data to a DataFrame and set the index to the datetime
data = pd.DataFrame(data)
data['submission_time'] = pd.to_datetime(data['submission_time'])
data = data.set_index('submission_time')

# Number of future periods to predict
num_future_periods = 5

def predict_future_data(data, num_periods):
    X = data.index.values.reshape(-1, 1)
    y_views = data['Views']
    y_comments = data['Comments']

    future_views = []
    future_comments = []

    for _ in range(num_periods):
        model_views = LinearRegression()
        model_views.fit(X, y_views)
        future_views.append(model_views.predict([[X[-1][0] + 1]]))
        y_views = pd.concat([y_views, pd.Series(future_views[-1])], ignore_index=True)

        model_comments = LinearRegression()
        model_comments.fit(X, y_comments)
        future_comments.append(model_comments.predict([[X[-1][0] + 1]]))
        y_comments = pd.concat([y_comments, pd.Series(future_comments[-1])], ignore_index=True)

        X = pd.concat([pd.DataFrame(X), pd.DataFrame([[X[-1][0] + 1]])], ignore_index=True)

    return y_views, y_comments

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

def create_line_chart(tab):
    if tab == 'Reddit':
        data_to_plot = data
    elif tab == 'Source2':
        data_to_plot = data_source2

    future_views, future_comments = predict_future_data(data_to_plot, num_future_periods)

    future_dates = pd.date_range(start=data_to_plot.index[-1], periods=num_future_periods+1, closed='right')

    fig = px.line(data_to_plot, x=data_to_plot.index, y=['Views', 'Comments'], title='Views and Comments Over Time')

    fig.add_trace(go.Scatter(x=future_dates, y=future_views, mode='lines', name='Future Views'))
    fig.add_trace(go.Scatter(x=future_dates, y=future_comments, mode='lines', name='Future Comments'))

    fig.update_layout(
        xaxis_title='Upload Time',
        yaxis_title='Count',
        template='plotly_dark',
    )

    return dcc.Graph(figure=fig, style={'height': '72vh'})

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(html.Div([
            html.H1("Pizza Hut Social Media Data Dashboard", style={'color': 'white', 'background-color': 'black'}),
            create_multi_bar_chart(),
        ]), width=6, style={'padding': '20px', 'background-color': 'grey'}),
        dbc.Col([
            dcc.Tabs(id='tabs', value='Reddit', children=[
                dcc.Tab(label='Reddit', value='Reddit', style={'color': 'white', 'background-color': 'black', 'font-size': '25px'}),
                dcc.Tab(label='Source2', value='Source2', style={'color': 'white', 'background-color': 'black', 'font-size': '25px'}),
            ]),
            html.Div(id='tabs-content', style={'background-color': 'grey'}),
        ], width=6, style={'padding': '20px', 'background-color': 'grey'}),
    ]),
])

@app.callback(
    Output('tabs-content', 'children'),
    [Input('tabs', 'value')]
)
def render_content(tab):
    return create_line_chart(tab)

if __name__ == '__main__':
    app.run_server(debug=True)


NameError: name 'create_multi_bar_chart' is not defined